# Using Multiple Correspondence Analysis to understand the relationships among characteristics of notable people

Multiple Correspondence Analysis (MCA) is a type of biplot for categorical data. Here we use it to understand the relationships among nominal (categorical) characteristics of notable people.  These data can be thought of as a 4-way contingency table (sex x occupation x birth era x region).  The goal of MCA is to visualize the structure of this contingency table as a graph.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import prince
import matplotlib.pyplot as plt

Modify the path below to point to the data file as needed.

In [ ]:
pa = Path("/home/kshedden/data/Teaching/bhht")

The entire dataset is around 2.3 million rows.  You can restrict to the first million rows when exploring, then switch to the whole dataset for final analyses.

In [ ]:
ca = ["birth", "death", "gender", "un_region", "level1_main_occ", "name"]
df = pd.read_csv(pa / Path("cross-verified-database.csv.gz"), usecols=ca, encoding="latin-1", nrows=1000000)

Rename the variables so that they fit on the plot.

In [ ]:
df = df.rename({"level1_main_occ": "occ", "gender": "sex", "un_region": "reg"}, axis=1)
df = df[["birth", "occ", "sex", "reg", "name"]].dropna()

Since very few people in the dataset lived priort to 1500 we exclude them here.

In [ ]:
df = df.query("birth >= 1500")

Remove very infrequent or difficult to interpret categories

In [ ]:
df = df.loc[df.occ != "Other", :]
df = df.loc[df.occ != "Missing", :]
df = df.loc[df.sex != "Other", :]

Create a century of birth variable, make it a string so that it is interpreted as a nominal variable.

In [ ]:
df.loc[:, "era"] = df.birth.round(-2)
df = df.drop("birth", axis=1)
df.loc[:, "era"] = ["%d" % x for x in df.era]
df.head()

The goal of MCA is to visualize the contingency table below.

In [ ]:
df.groupby(["sex", "reg", "occ", "era"]).count().unstack().fillna(0)

Fit multiple correspondence analysis

In [ ]:
mca = prince.MCA(n_components=4)
df = df.drop(columns=["name"])
mca = mca.fit(df)
mca.transform(df)

In [ ]:
cols = {"occ": "orange", "sex": "purple", "reg": "lime", "era": "navy"}

Plot the category scores.  There are so many object scores that a scatterplot of object scores would be difficult to interpret.

In [ ]:
def mca_plot(mca, cols, jx, jy):
    cc = mca.column_coordinates(df)
    xmin, xmax = cc.iloc[:, jx].min(), cc.iloc[:, jx].max()
    d = xmax - xmin
    xmin -= 0.1*d
    xmax += 0.1*d
    ymin, ymax = cc.iloc[:, jy].min(), cc.iloc[:, jy].max()
    d = ymax - ymin
    ymin -= 0.1*d
    ymax += 0.1*d

    plt.clf()
    plt.grid(True)
    for k in cols.keys():
        cx = cc[cc.index.str.startswith(k)]
        if k == "era":
            plt.plot(cx.iloc[:, jx], cx.iloc[:, jy], "-", color=cols[k])
        for i in range(cx.shape[0]):
            plt.text(cx.iloc[i, jx], cx.iloc[i, jy], cx.index[i], color=cols[k],
                     ha="center", va="center")
    plt.xlabel("Component %d" % (jx + 1))
    plt.ylabel("Component %d" % (jy + 1))
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

In [ ]:
mca_plot(mca, cols, 0, 1)

In [ ]:
mca_plot(mca, cols, 2, 3)

In an MCA plot, information about the variables is encoded both in the angles between variable scores, and in the magnitudes of the variable scores.  Variables that are uncorrelated with all other variables have very short magnitudes.  To demonstrate this, we create a variable that is independent of the others and include it in the MCA.

In [ ]:
df["fake"] = np.random.choice([0, 1], df.shape[0])
cols1 = cols.copy()
cols1["fake"] = "red"

mca1 = prince.MCA(n_components=4)
mca1 = mca1.fit(df)
mca1.transform(df)
mca_plot(mca1, cols1, 0, 1)